In [1]:
import random
import numpy as np
import copy

In [2]:
class Chromosome(object):
    """ Chromosome class that encapsulates an individual's \
        fitness and solution representation.
    """
    def __init__(self, genes):
        """Initialise the Chromosome."""
        self.genes = genes
        self.fitness = 0
    def __repr__(self):
        """ Return initialised Chromosome representation in human \
            readable form.
        """
        return repr((self.fitness, self.genes))
class JSSP(object):
    def __init__(self,J,M,times,ordering_machines,
                 population_size,generations,
                 crossover_probability,mutation_probability,
                 tournament_size,elitism):
        self.J = J
        self.M = M
        self.N = J*M
        self.times = times
        self.positions = list(np.arange(0,self.N))
        self.ordering_machines = ordering_machines
        self.seed_data = list(np.arange(1,self.J+1))
        self.population_size = population_size
        self.generations = generations
        self.crossover_probability = crossover_probability
        self.mutation_probability = mutation_probability
        self.tournament_size = tournament_size
        self.elitism = elitism
        self.current_generation = []
        self.minimize_fitness = False
        
        def assignment_machines(list_jobs):   
            times = np.zeros(self.J,dtype=int)
            list_machines = np.zeros(self.N,dtype=int)    
            for j,job in enumerate(list_jobs):
                list_machines[j] = self.ordering_machines[job-1][times[job-1]]
                times[job-1] += 1
            return list_machines
        
        def fitness(individual):
            list_machines = assignment_machines(individual)         
            free = np.zeros(self.M)
            final = np.zeros(self.N)-1
            present = 0
            while -1 in final:
                for j in self.positions:
                    job = individual[j]
                    if final[j] != -1:
                        continue
                    machine = list_machines[j]
                    if present < free[machine-1]:
                        continue
                    ready = True
                    elapsed = 0
                    for i in range(0,j):
                        cond1 = individual[i]==job and (final[i]>present or final[i]==-1)
                        cond2 = list_machines[i]==machine and final[i]==-1
                        if cond1 or cond2:
                            ready = False
                            break
                    if ready:
                        for i in range(j-1, -1, -1):
                            if individual[i] == job:
                                elapsed = final[i]
                                break
                        final[j] = max(free[machine-1], elapsed) + times[job-1][machine-1]
                        free[machine-1] = final[j]
                    else:
                        continue
                i = 0
                past = present
                while free[i] <= present:
                    i += 1
                present = free[i]
                i += 1
                while i < self.M:
                    if free[i]>past and free[i]<present:
                        present = free[i]
                    i += 1
            return max(free)

        def ordering_jobs(self,list_jobs, list_machines):
            times = np.zeros(self.M, dtype=int)
            M_J = np.zeros((self.M,self.J), dtype=int)
            for m in self.positions:
                temp = list_machines[m] -1
                M_J[temp][times[temp]] = list_jobs[m]
                times[temp] +=1
            return M_J

        def mutate(individual):
            index_1,index_2 = random.sample(self.positions,2)
            individual[index_1], individual[index_2] = individual[index_2], individual[index_1]
        
        def delete_job(list_jobs,job,repeats,p1,p2):
            posi = []
            n = len(list_jobs)
            j = 0
            for i in range(0,n):
                if job == list_jobs[i] and (i<p1 or i>p2 ) :
                    posi.append(i)
            posi = random.sample(posi,repeats)
            save_p1 = p1
            for i in posi:
                if i < save_p1:
                    p1 -= 1
                    p2 -= 1
                list_jobs[i] = -1
            for _ in range(0,repeats):
                list_jobs.remove(-1)
            return p1,p2
    
        def delete(list_jobs,p1,p2):
            for i in self.seed_data:
                repeats = 0
                for job in list_jobs:
                    if job == i:
                        repeats += 1
                repeats = repeats - self.M
                if repeats > 0:
                    p1,p2 = delete_job(list_jobs,i,repeats,p1,p2)
            return p1,p2
        
        def complete_job(list_jobs,job,repeats,p1,p2):
            n = len(list_jobs)
            for _ in range(0,repeats):
                randi = random.randint(0,n)
                while randi>p1 and randi<=p2:
                    randi = random.randint(0,n)
                if randi <= p1:
                    p1 += 1
                    p2 += 1
                list_jobs.insert(randi,job)
                n += 1
            return p1,p2
        
        def complete(list_jobs,p1,p2):
            for i in self.seed_data:
                repeats = 0
                for job in list_jobs:
                    if job == i:
                        repeats += 1
                repeats = self.M - repeats
                if repeats > 0:
                    p1,p2 = complete_job(list_jobs,i,repeats,p1,p2)
            return p1,p2
        
        def crossover(parent_1,parent_2):
            index_1 = random.sample(self.positions,2)
            index_2 = random.sample(self.positions,2)
            index_1.sort()
            index_2.sort()
            cross_11,cross_12 = index_1
            cross_21,cross_22 = index_2
            child_1a = parent_1[:cross_11]
            child_1b = parent_2[cross_21:cross_22+1]
            child_1c = parent_1[cross_12+1:]
            child_2a = parent_2[:cross_21]
            child_2b = parent_1[cross_11:cross_12+1]
            child_2c = parent_2[cross_22+1:]
            child_1 = child_1a + child_1b + child_1c
            child_2 = child_2a + child_2b + child_2c
            jump_1 = cross_22 - cross_21
            jump_2 = cross_12 - cross_11
            cross_11 = len(child_1a)
            cross_12 = cross_11 + jump_1
            cross_21 = len(child_2a)
            cross_22 = cross_21 + jump_2
            cross_11,cross_12 = delete(child_1,cross_11,cross_12)
            cross_11,cross_12 = complete(child_1,cross_11,cross_12)
            cross_21,cross_22 = delete(child_2,cross_21,cross_22) 
            cross_21,cross_22 = complete(child_2,cross_21,cross_22)
            return child_1, child_2

        def selection(population):
            members = random.sample(population,self.tournament_size)
            members.sort(key=lambda x: x.fitness,reverse=self.minimize_fitness)
            return members[0]

        def create_individual(data):
            individual = self.positions[:]
            pos = self.positions[:]
            for job in data:
                in_machine = random.sample(pos,self.M)
                for machine in in_machine:
                    individual[machine] = job
                    pos.remove(machine)
            return individual
        
        self.fitness = fitness
        self.selection = selection
        self.create_individual = create_individual
        self.mutate = mutate
        self.crossover = crossover
        
    def create_initial_population(self):
        initial_population = []
        for _ in range(self.population_size):
            genes = self.create_individual(self.seed_data)
            individual = Chromosome(genes)
            initial_population.append(individual)
        self.current_generation = initial_population

    def calculate_population_fitness(self):
        for individual in self.current_generation:
            individual.fitness = self.fitness(individual.genes)

    def rank_population(self):
        self.current_generation.sort(key=lambda x: x.fitness,
                                     reverse=self.minimize_fitness)

    def create_new_population(self):
        new_population = []
        elite = copy.deepcopy(self.current_generation[0])
        while len(new_population) < self.population_size:
            parent_1 = copy.deepcopy(self.selection(self.current_generation))
            parent_2 = copy.deepcopy(self.selection(self.current_generation))
            child_1, child_2 = parent_1, parent_2
            child_1.fitness, child_2.fitness = 0, 0
            can_crossover = random.random() < self.crossover_probability
            can_mutate = random.random() < self.mutation_probability
            if can_crossover:
                child_1.genes, child_2.genes = self.crossover(parent_1.genes,
                                                              parent_2.genes)
            if can_mutate:
                self.mutate(child_1.genes)
                self.mutate(child_2.genes)
            new_population.append(child_1)
            if len(new_population) < self.population_size:
                new_population.append(child_2)
        if self.elitism:
            new_population[0] = elite
        self.current_generation = new_population

    def create_first_generation(self):
        self.create_initial_population()
        self.calculate_population_fitness()
        self.rank_population()

    def best_individual(self):
        best = self.current_generation[0]
        return (best.fitness, best.genes)

    def create_next_generation(self):
        self.create_new_population()
        self.calculate_population_fitness()
        self.rank_population()

    def run(self):
        self.create_first_generation()
        for _ in range(1, generations):
            self.create_next_generation()

    def last_generation(self):
        return ((member.fitness, member.genes) for member in self.current_generation)

In [16]:
J = 6  # Number of jobs
M = 6  # Number o machines
population_size = 100
generations = 300
crossover_probability = 0.8
mutation_probability = 0.2
tournament_size = 4
elitism = True
times = [
    [3,6,1,7,6,3],
    [10,8,5,4,10,10],
    [9,1,5,4,7,8],
    [5,5,5,3,8,9],
    [3,3,9,1,5,4],
    [10,3,1,3,4,9]
    ]
ordering_machines = [
    [3,1,2,4,6,5],
    [2,3,5,6,1,4],
    [3,4,6,1,2,5],
    [2,1,3,4,5,6],
    [3,2,5,6,1,4],
    [2,4,6,1,5,3],
    ]
problem = JSSP(J,M,times,ordering_machines,population_size,
               generations,crossover_probability,
               mutation_probability,tournament_size,elitism)
for _ in range(0,5):
    problem.run()
    print(problem.best_individual())

(55.0, [1, 2, 3, 4, 1, 3, 6, 2, 2, 5, 4, 3, 4, 5, 1, 6, 6, 3, 2, 4, 5, 5, 1, 4, 6, 2, 3, 6, 3, 5, 1, 1, 2, 4, 5, 6])
(55.0, [1, 1, 3, 2, 3, 4, 2, 5, 6, 4, 2, 4, 5, 5, 6, 4, 4, 3, 1, 6, 2, 3, 1, 6, 6, 5, 3, 2, 6, 3, 1, 5, 1, 2, 4, 5])
(55.0, [1, 2, 3, 2, 6, 1, 2, 4, 3, 3, 1, 5, 6, 4, 4, 3, 6, 4, 1, 5, 6, 3, 2, 1, 5, 4, 3, 2, 5, 5, 2, 6, 1, 6, 4, 5])
(55.0, [1, 3, 3, 1, 2, 2, 5, 4, 4, 6, 6, 3, 2, 5, 6, 3, 4, 3, 5, 2, 1, 4, 1, 5, 4, 6, 6, 3, 2, 1, 1, 6, 5, 2, 5, 4])
(55.0, [2, 1, 3, 3, 1, 2, 4, 6, 3, 5, 4, 5, 6, 6, 3, 6, 1, 2, 4, 5, 4, 4, 2, 1, 1, 6, 5, 4, 2, 3, 6, 3, 5, 1, 2, 5])


[2, 3, 2, 3, 2, 3, 1, 1, 1]
[2, 3, 2, 1, 2, 3, 1, 1, 3]


4